# Collision Avoidance - Live Demo

In this notebook we'll use the model we trained to detect whether the robot is ``free`` or ``blocked`` to enable a collision avoidance behavior on the robot.  

## Load the trained model

We'll assumed that you've already downloaded the ``best_model.pth`` to your workstation as instructed in the training notebook.  Now, you should upload this model into this notebook's
directory by using the Jupyter Lab upload tool.  Once that's finished there should be a file named ``best_model.pth`` in this notebook's directory.  

> Please make sure the file has uploaded fully before calling the next cell

Execute the code below to initialize the PyTorch model.  This should look very familiar from the training notebook.

In [1]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

Next, load the trained weights from the ``best_model.pth`` file that you uploaded

In [2]:
model.load_state_dict(torch.load('best_model.pth'))

<All keys matched successfully>

Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.

In [3]:
device = torch.device('cuda')
model = model.to(device)

### Create the preprocessing function

We have now loaded our model, but there's a slight issue.  The format that we trained our model doesn't *exactly* match the format of the camera.  To do that, 
we need to do some *preprocessing*.  This involves the following steps

1. Convert from BGR to RGB
2. Convert from HWC layout to CHW layout
3. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
4. Transfer the data from CPU memory to GPU memory
5. Add a batch dimension

In [4]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

Great! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

Now, let's start and display our camera.  You should be pretty familiar with this by now.  We'll also create a slider that will display the
probability that the robot is blocked.  We'll also display a slider that allows us to control the robot's base speed.

In [ ]:
import cv2
import numpy as np
import traitlets
import ipywidgets as widgets
from IPython.display import display
import time
import threading

class ImageHolder(traitlets.HasTraits):
    value = traitlets.Bytes()

class OpenCVCamera(traitlets.HasTraits):
    image = traitlets.Bytes()

    def __init__(self, device_id=0, width=640, height=480, fps=30, **kwargs):
        super().__init__(**kwargs)
        self.device_id = device_id
        self.width = width
        self.height = height
        self.fps = fps
        self.cap = cv2.VideoCapture(self.device_id)

        if not self.cap.isOpened():
            raise RuntimeError(f"Failed to open camera with device ID {self.device_id}")

        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.width)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.height)
        self.cap.set(cv2.CAP_PROP_FPS, self.fps)

    def update(self):
        """Capture a new image from the camera and update the `image` trait."""
        ret, frame = self.cap.read()
        if not ret:
            print(f"Failed to capture image from device {self.device_id}")
            return

        try:
            # Attempt to encode the frame
            ret, jpeg = cv2.imencode('.jpg', frame)
            if ret:
                self.image = jpeg.tobytes()
            else:
                print("Failed to encode image")
        except cv2.error as e:
            print(f"OpenCV error during image encoding: {e}")

    def release(self):
        """Release the camera resource."""
        if self.cap is not None:
            self.cap.release()
            print("Camera released")

    def __del__(self):
        """Destructor to ensure resources are released when object is deleted."""
        self.release()

def release_camera(camera):
    """Function to release camera and related resources."""
    camera.release()
    cv2.destroyAllWindows()
    print("Resources cleaned up")

def update_widget(change):
    if change['new']:
        image_widget.value = change['new']
    else:
        print("Image data is empty or invalid")

def run_camera(camera):
    """Function to run the camera in a separate thread."""
    try:
        while True:
            camera.update()
            time.sleep(1 / camera.fps)  # Sleep to match the camera's fps
    except KeyboardInterrupt:
        print("Interrupted by user")
    finally:
        release_camera(camera)

# Initialize the camera
try:
    camera = OpenCVCamera(width=640, height=480)
except RuntimeError as e:
    print(f"Error initializing camera: {e}")
    camera = None

if camera:
    image_holder = ImageHolder()

    # Link the camera image to the image_holder value directly
    camera_link = traitlets.dlink((camera, 'image'), (image_holder, 'value'))

    # Create an image widget to display the camera feed
    image_widget = widgets.Image()
    image_holder.observe(update_widget, names='value')

    # Sliders for "blocked" and "speed" control
    blocked_slider = widgets.FloatSlider(description='Blocked', min=0.0, max=1.0, orientation='vertical')
    speed_slider = widgets.FloatSlider(description='Speed', min=0.0, max=0.5, value=0.0, step=0.01, orientation='horizontal')

    # Display the image and sliders
    display(widgets.VBox([widgets.HBox([image_widget, blocked_slider]), speed_slider]))

    # Run the camera in a separate thread
    camera_thread = threading.Thread(target=run_camera, args=(camera,))
    camera_thread.start()


We'll also create our robot instance which we'll need to drive the motors.

In [9]:
from jetbot import Robot

robot = Robot()

OSError: [Errno 5] Input/output error

Next, we'll create a function that will get called whenever the camera's value changes.  This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network
3. While the neural network output indicates we're blocked, we'll turn left, otherwise we go forward.

In [ ]:
import torch.nn.functional as F
import time

def preprocess(image_bytes):
    """Convert image bytes to a tensor and preprocess."""
    # Example preprocessing code; adjust as necessary
    image_array = np.frombuffer(image_bytes, dtype=np.uint8).reshape((224, 224, 3))
    image_tensor = torch.tensor(image_array).permute(2, 0, 1).float() / 255.0
    image_tensor = image_tensor.unsqueeze(0)  # Add batch dimension
    return image_tensor

def update(change):
    global blocked_slider, speed_slider, model, robot
    image_bytes = change['new']
    if image_bytes:
        x = preprocess(image_bytes)
        y = model(x)
        
        # Apply the softmax function to normalize the output vector
        y = F.softmax(y, dim=1)
        
        prob_blocked = float(y.flatten()[0])
        
        blocked_slider.value = prob_blocked
        
        if prob_blocked < 0.5:
            robot.forward(speed_slider.value)
        else:
            robot.left(speed_slider.value)
        
        time.sleep(0.001)

# Make sure `camera.image` is properly linked to `update`
if camera:
    image_holder = ImageHolder()
    camera_link = traitlets.dlink((camera, 'image'), (image_holder, 'value'))
    image_holder.observe(update, names='value')

    # Initialize sliders and robot as needed
    blocked_slider = widgets.FloatSlider(description='Blocked', min=0.0, max=1.0, orientation='vertical')
    speed_slider = widgets.FloatSlider(description='Speed', min=0.0, max=0.5, value=0.0, step=0.01, orientation='horizontal')

    display(widgets.VBox([widgets.HBox([blocked_slider]), speed_slider]))
    
    # Example initialization (ensure `model` and `robot` are defined)
    # model = ... (your model loading code here)
    # robot = ... (your robot initialization code here)
    
    # Start the camera feed in a separate thread
    camera_thread = threading.Thread(target=run_camera, args=(camera,))
    camera_thread.start()


In [12]:
import cv2
import numpy as np
import torch
import torch.nn.functional as F
import traitlets
import ipywidgets as widgets
from IPython.display import display
import time
import threading

class ImageHolder(traitlets.HasTraits):
    value = traitlets.Bytes()

class OpenCVCamera(traitlets.HasTraits):
    image = traitlets.Bytes()

    def __init__(self, device_id=0, width=640, height=480, fps=30, **kwargs):
        super().__init__(**kwargs)
        self.device_id = device_id
        self.width = width
        self.height = height
        self.fps = fps
        self.cap = cv2.VideoCapture(self.device_id)

        if not self.cap.isOpened():
            raise RuntimeError(f"Failed to open camera with device ID {self.device_id}")

        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.width)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.height)
        self.cap.set(cv2.CAP_PROP_FPS, self.fps)

    def update(self):
        """Capture a new image from the camera and update the `image` trait."""
        ret, frame = self.cap.read()
        if not ret:
            print(f"Failed to capture image from device {self.device_id}")
            return

        try:
            # Encode frame to JPEG
            ret, jpeg = cv2.imencode('.jpg', frame)
            if ret:
                self.image = jpeg.tobytes()
            else:
                print("Failed to encode image")
        except cv2.error as e:
            print(f"OpenCV error during image encoding: {e}")

    def release(self):
        """Release the camera resource."""
        if self.cap is not None:
            self.cap.release()
            print("Camera released")

    def __del__(self):
        """Destructor to ensure resources are released when object is deleted."""
        self.release()

def release_camera(camera):
    """Function to release camera and related resources."""
    camera.release()
    cv2.destroyAllWindows()
    print("Resources cleaned up")

def preprocess(image_bytes):
    """Convert image bytes to a tensor and preprocess."""
    # Convert bytes to a numpy array
    nparr = np.frombuffer(image_bytes, np.uint8)
    
    # Decode the numpy array into an image
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    
    if img is None:
        raise ValueError("Failed to decode image")
    
    # Resize the image if needed
    img = cv2.resize(img, (224, 224))
    
    # Convert to tensor
    image_tensor = torch.tensor(img).permute(2, 0, 1).float() / 255.0
    image_tensor = image_tensor.unsqueeze(0)  # Add batch dimension
    return image_tensor

def update(change):
    global blocked_slider, speed_slider, model, robot
    image_bytes = change['new']
    if image_bytes:
        x = preprocess(image_bytes)
        y = model(x)
        
        # Apply the softmax function to normalize the output vector
        y = F.softmax(y, dim=1)
        
        prob_blocked = float(y.flatten()[0])
        
        blocked_slider.value = prob_blocked
        
        if prob_blocked < 0.5:
            robot.forward(speed_slider.value)
        else:
            robot.left(speed_slider.value)
        
        time.sleep(0.001)

# Initialize the camera
try:
    camera = OpenCVCamera(width=640, height=480)
except RuntimeError as e:
    print(f"Error initializing camera: {e}")
    camera = None

if camera:
    image_holder = ImageHolder()
    
    # Link the camera image to the image_holder value directly
    camera_link = traitlets.dlink((camera, 'image'), (image_holder, 'value'))
    image_holder.observe(update, names='value')

    # Create an image widget to display the camera feed
    image_widget = widgets.Image()
    image_widget.observe(update, names='value')
    
    # Sliders for "blocked" and "speed" control
    blocked_slider = widgets.FloatSlider(description='Blocked', min=0.0, max=1.0, orientation='vertical')
    speed_slider = widgets.FloatSlider(description='Speed', min=0.0, max=0.5, value=0.0, step=0.01, orientation='horizontal')

    # Display the image and sliders
    display(widgets.VBox([widgets.HBox([image_widget, blocked_slider]), speed_slider]))
    
    # Example initialization (ensure `model` and `robot` are defined)
    # model = ... (your model loading code here)
    # robot = ... (your robot initialization code here)
    
    # Run the camera in a separate thread
    camera_thread = threading.Thread(target=run_camera, args=(camera,))
    camera_thread.start()


Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0


Failed to capture image from device 0
Failed to capture image from device 0Failed to capture image

Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0Failed to capture image from device 0

Failed to capture image
Failed to capture image from device 0Failed to capture image from device 0

Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0


Exception in thread Thread-8:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-2ca60b825834>", line 73, in run_camera
    camera.update()
  File "<ipython-input-12-00a625d173f8>", line 43, in update
    self.image = jpeg.tobytes()
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 588, in __set__
    self.set(obj, value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 577, in set
    obj._notify_trait(self.name, old_value, new_value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1210, in _notify_trait
    type='change',
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1215, in notify_change
    return self._notify_observers(change)
  File "/usr/local/lib/python3.6/dist-packa

Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0


Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing. 

We accomplish that with the ``observe`` function.

> WARNING: This code may move the robot!! Adjust the speed slider we defined earlier to control the base robot speed.  Some kits can move fast, so start slow, and gradually increase the value.

In [14]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

Failed to capture image
Failed to capture image from device 0Failed to capture image from device 0

Failed to capture imageFailed to capture image from device 0Failed to capture image from device 0


Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0
Failed to capture image
Failed to capture image from device 0
Failed to capture image from device 0


Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame.  Perhaps start by placing your robot on the ground and seeing what it does when it reaches an obstacle.

If you want to stop this behavior, you can unattach this callback by executing the code below.

In [ ]:
import time

camera.unobserve(update, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

Perhaps you want the robot to run without streaming video to the browser.  You can unlink the camera as below.

In [ ]:
camera_link.unlink()  # don't stream to browser (will still run camera)

To continue streaming call the following.

In [ ]:
camera_link.link()  # stream to browser (wont run camera)

Again, let's close the camera conneciton properly so that we can use the camera in other notebooks.

In [ ]:
camera.stop()

### Conclusion

That's it for this live demo!  Hopefully you had some fun and your robot avoided collisions intelligently! 

If your robot wasn't avoiding collisions very well, try to spot where it fails.  The beauty is that we can collect more data for these failure scenarios
and the robot should get even better :)